In [1]:
import numpy as np
import pandas as pd

from dataset import get_dataset
from trainer import Trainer
from inference import create_label, sum_label

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models

In [ ]:
## hyperparameters
lr = 0.00003
batch_size = 64
num_epoch = 30
target = 'label'

In [3]:
## prepare dataset

df_train = pd.read_csv('df_all.csv')
#df_train = pd.read_csv('df_train.csv')
df_valid = pd.read_csv('df_valid.csv')
df_test = pd.read_csv('/opt/ml/input/data/eval/info.csv')

train_dataset, valid_dataset, test_dataset = get_dataset(df_train, df_valid, df_test, target=target)

In [4]:
target = train_dataset.target
weight = df_train[target].value_counts().sort_index().to_numpy()
print('count :', weight)
weight = 1. / weight
samples_weight = np.array([weight[t] for t in df_train[target]])
samples_weight = torch.from_numpy(samples_weight)

count : [2745 1570  895 3660 3345 1285  549  314  179  732  669  257  549  314
  179  732  669  257]


In [5]:
from torch.utils.data.sampler import WeightedRandomSampler
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight)//2)

In [6]:
## prepare model (resnet)

device = 'cuda'
model = models.resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
## prepare model (efficient_net)

from efficientnet_pytorch import EfficientNet

device = 'cuda'
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=len(train_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Loaded pretrained weights for efficientnet-b0


In [7]:
## prepare Dataloader

dataloaders = {'train' : DataLoader(train_dataset, batch_size=batch_size, num_workers=3, drop_last=True, sampler=sampler),
               'valid' : DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=3, drop_last=False)}
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=3, drop_last=False)

In [8]:
## train

trainer = Trainer('/opt/ml/code/save')

In [ ]:
# change lr manually:

# for g in optim.param_groups:
#     g['lr'] = 0.001

In [12]:
#model.load_state_dict(torch.load('/opt/ml/code/save/label/effnet_test018.pt'))

trainer.train(model, dataloaders, criterion, optimizer, device, num_epochs=num_epoch, scheduler=None, save_name='effnet_test', sub_dir=target)

Epoch 1/30
----------


100%|██████████| 147/147 [00:38<00:00,  3.81it/s]


train Loss: 0.1771 Acc: 0.9374 F1: 0.9375016756797544


100%|██████████| 74/74 [00:09<00:00,  7.61it/s]


valid Loss: 0.0201 Acc: 0.9953 F1: 0.9960286941481069

Epoch 2/30
----------


...

100%|██████████| 147/147 [00:38<00:00,  3.80it/s]


train Loss: 0.1320 Acc: 0.9528 F1: 0.952827951213968


100%|██████████| 74/74 [00:09<00:00,  7.70it/s]


valid Loss: 0.0068 Acc: 0.9985 F1: 0.9985736080432789

Training complete!


NameError: name 'best_acc' is not defined

In [13]:
## inference

model.load_state_dict(torch.load('/opt/ml/code/save/label/effnet_test028.pt'))
df_submit = create_label(model, test_dataloader, df_test.copy(), device)

# model.load_state_dict(torch.load('/opt/ml/code/save/age/6e5_9110.pt'))
# df_submit = create_label(model, test_dataloader, df_test.copy(), device, target='age')

# model.load_state_dict(torch.load('/opt/ml/code/save/mask/6e5_9987.pt'))
# df_submit = create_label(model, test_dataloader, df_submit, device, target='mask')

# model.fc = nn.Linear(model.fc.in_features, 2).cuda()
# model.load_state_dict(torch.load('/opt/ml/code/save/gender/6e5_9851.pt'))
# df_submit = create_label(model, test_dataloader, df_submit, device, target='gender')

# df_submit = sum_label(df_submit)

df_submit.to_csv('submit.csv', index=False)

100%|██████████| 197/197 [00:24<00:00,  7.92it/s]

inference ans complete!


In [ ]:
from utils import imshow
batch = next(iter(dataloaders['train']))
imshow(batch[0])
print(batch[1])

In [ ]:
next(iter(dataloaders['train']))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

model.eval()
y_true = []
y_pred = []
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(preds.tolist())

mtx = confusion_matrix(y_true, y_pred)
print(mtx)


In [ ]:
import matplotlib.pyplot as plt

label_ = list(range(18))

fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(mtx, cmap='Blues', vmax=100, vmin=-20)

ax.set_xticks(np.arange(len(label_)))
ax.set_yticks(np.arange(len(label_)))
# ... and label them with the respective list entries
ax.set_xticklabels(label_)
ax.set_yticklabels(label_)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(label_)):
    for j in range(len(label_)):
        if mtx[i, j]:
            text = ax.text(j, i, mtx[i, j],
                           ha="center", va="center", color="w", size=10)
ax.set_xlabel('Prediction')
ax.set_ylabel('True Label')
ax.set_title("Confusion mtx")
fig.tight_layout()
plt.show()